In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype=torch.float)
labels = true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)

In [4]:
import torch.utils.data as Data
batch_size = 10
dataset = Data.TensorDataset(features,labels)
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

In [5]:
for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.3710, -0.1920],
        [-0.7039, -1.3354],
        [ 0.5104,  0.3418],
        [-0.1173,  1.2200],
        [ 0.8985, -3.6428],
        [ 1.8112, -1.4740],
        [ 1.0257,  1.0466],
        [-0.4103, -0.6097],
        [ 0.4988,  0.1116],
        [ 0.4376,  1.3456]]) tensor([ 5.6085,  7.3556,  4.0639, -0.1996, 18.3752, 12.8428,  2.6953,  5.4400,
         4.8078,  0.4833])


In [10]:
import torch.nn as nn

In [14]:
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_feature,1)
    def forward(self,x):
        y = self.linear(x)
        return y
net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [16]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.1717, -0.0286]], requires_grad=True)
Parameter containing:
tensor([-0.1016], requires_grad=True)


In [17]:
from torch.nn import init
init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)


TypeError: 'LinearNet' object is not subscriptable

In [18]:
loss = nn.MSELoss()

In [19]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [20]:
optimizer = optim.SGD([
    {'params':net.subnet1.parameters()},
    {'params':net.subnet2.parameters(),'lr':0.01}
],lr=0.03)

AttributeError: 'LinearNet' object has no attribute 'subnet1'

In [21]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

In [22]:
num_epochs = 3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output = net(X)
        l = loss(output,y.view(-1,1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d,loss:%f'%(epoch,l.item()))

epoch 1,loss:17.171190
epoch 2,loss:2.149895
epoch 3,loss:0.526969


In [23]:
dense = net[0]
print(true_w,dense.weight)
print(true_b,dense.bias)

TypeError: 'LinearNet' object is not subscriptable